In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
iop_phe <- fread(
    '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/iop/misc/ukb2005693_ukb37855_IOP.phe',
    stringsAsFactors=FALSE,
    data.table=FALSE
) %>%
mutate(
    FID = as.character(FID),
    IID = as.character(IID)    
)


In [6]:
glaucoma_phe <- fread(
    '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/highconfidenceqc/phe/HC276.phe',
    stringsAsFactors=FALSE,
    data.table=FALSE,
    col.names=c('FID', 'IID', 'HC276')
) %>%
mutate(
    FID = as.character(FID),
    IID = as.character(IID)    
)


In [8]:
glaucoma_phe %>% count(HC276)

HC276,n
<int>,<int>
1,479760
2,8614


In [15]:
phe_all <- iop_phe %>% 
full_join(glaucoma_phe %>% mutate(HC276 = HC276 - 1), by=c('FID', 'IID'))


In [16]:
iop_phe      %>% dim() %>% print()
glaucoma_phe %>% dim() %>% print()
phe_all      %>% dim() %>% print()


[1] 128943      8
[1] 488374      3
[1] 492305      9


In [20]:
phe_all %>% colnames() %>% data.frame() %>%
fwrite('private_data/IOP_glaucoma.phe.colnames.txt', row.names = FALSE, col.names = FALSE)


In [23]:
phe_all %>% 
fwrite(
    'private_data/IOP_glaucoma.without.colnames.phe', 
    row.names = FALSE, col.names = FALSE, sep='\t',
    quote = FALSE, na='NA'
)


In [24]:
phe_all %>% 
fwrite(
    'private_data/IOP_glaucoma.phe', 
    row.names = FALSE, col.names = TRUE, sep='\t',
    quote = FALSE, na='NA'
)


## counts

In [25]:
pop_def <- bind_rows(lapply(c('white_british', 'non_british_white', 'e_asian', 's_asian', 'african'), function(pop){
    fread(
        file.path(
            '/oak/stanford/groups/mrivas/ukbb24983',
            'sqc/population_stratification', 
            paste0('ukb24983_', pop, '.phe')
        ),
        stringsAsFactors=FALSE,
        data.table=FALSE,
        colClasses = "character",
        col.names=c('FID', 'IID')
    ) %>%
    mutate(pop = pop)    
    }
))


In [4]:
pop_def%>% count(pop) %>% arrange(-n)


pop,n
<chr>,<int>
white_british,337151
non_british_white,24909
s_asian,7885
african,6498
e_asian,1154


In [6]:
iop_phe %>% 
inner_join(pop_def, by=c('FID', 'IID')) %>%
count(pop) %>%
arrange(-n)

pop,n
<chr>,<int>
white_british,82253
non_british_white,7892
s_asian,3672
african,3294
e_asian,408


In [32]:
iop_phe %>% 
inner_join(pop_def, by=c('FID', 'IID')) %>%
dim()

[1] 97519     9

In [34]:
c(
    exp(log(0.665432) - 1.96 * 0.148821),
    exp(log(0.665432) + 1.96 * 0.148821)
)

[1] 0.4970782 0.8908051